# Spectral Mesh Flattening

by Ana + Gabrielle

In [1]:
import trimesh
import numpy as np
import scipy
from PIL import Image

In [4]:
def boundary(mesh):
    
    """ A function to find the boundary edges and vertices of a mesh. 
        Inputs:
        mesh: a trimesh mesh. 
        Outputs: 
        boundary_vertices: a list of the indices of vertices on the boundary. 
        next_vertex: a dictionary to find the next vertex on the boundary. 
    """

    next_vertex = {}
    boundary_vertices = []
    boundary_edges = []
    start = []

    edges = mesh.edges
    vertex_faces = mesh.vertex_faces
    
    for i in range(len(edges)):

        edge = edges[i]
        v1, v2 = edge
        faces = [j for j in vertex_faces[v1] if j != -1 and j in vertex_faces[v2]]
        # Boundary edges are edges that are only in one face
        if len(faces) == 1:

            boundary_vertices = boundary_vertices + [v1, v2]
            if v1 not in start:
                boundary_edges = boundary_edges + [[v1, v2]]
                next_vertex[v1] = v2
            else:
                boundary_edges = boundary_edges + [[v1, v2]]
                next_vertex[v2] = v1
            
            start = start + boundary_edges[-1][0]
    
    next_v = {}
    for vert in boundary_vertices:

        first_edge, second_edge = [i for i in boundary_edges if vert in i]
        v1 = [i for i in first_edge if i != vert][0]
        v2 = [i for i in second_edge if i != vert][0]
    
        next_v[v1] = vert
        next_v[vert] = v2


    # Remove any duplicates
    boundary_vertices = np.unique(np.array(boundary_vertices))
    return boundary_vertices, next_vertex, boundary_edges #next_v, boundary_edges#

In [5]:
def circle_boundary(mesh, centre = [0, 0]):

    """ A function to map the boundary of a mesh to a circle centered at the origin. 
        Inputs:
        mesh: a trimesh mesh. 
        centre: optional argument to change the centre of the circle. 
        
        Outputs: 
        new_boundary_values: an array of boundary values (2 dimensional) on the circle. 
    """

    vertices = mesh.vertices

    range_x = (np.max(vertices[:, 0]) - np.min(vertices[:, 0]))/2
    range_y = (np.max(vertices[:, 1]) - np.min(vertices[:, 1]))/2
    range_z = (np.max(vertices[:, 2]) - np.min(vertices[:, 2]))/2

    # Keep the circle roughly the same size as the mesh by choosing half the maximum range
    # as the radius 
    radius = np.max([range_x, range_y, range_z])

    # find the boundary information, and intialize the output 
    b_verts, next_vertex, _ = boundary(mesh)
    output = np.zeros((len(vertices), 2))
    weights = {}
    
    # Calculate the "weights" of each edge and store in a dictionary 
    # to be used to determine how far from the other vertices each edge 
    for i in range(len(b_verts)):
        weights[b_verts[i]] = np.linalg.norm(vertices[b_verts[i]] - vertices[next_vertex[b_verts[i]]])
    
    total = sum(weights.values(), 0.0)
    weights = {k: v / total for k, v in weights.items()}

    # Start with one point set at 0 degrees, and then increase the angle each time (since arc length is
    # proportional to angle)
    angle_sum = 0
    v1 = b_verts[0]
    output[v1] = [centre[0] + radius * np.cos(angle_sum), centre[1] + radius * np.sin(angle_sum)]

    for i in range(len(b_verts)):
        
        angle = weights[v1] * 2 * np.pi
        angle_sum += angle

        # calculate the new positions
        new_x = centre[0] + radius * np.cos(angle_sum)
        new_y = centre[1] + radius * np.sin(angle_sum)

        v1 = next_vertex[v1]
        output[v1] = [new_x, new_y]

    # make sure the outputs are in the same order as the b_verts 
    new_boundary_values = output[b_verts]

    return new_boundary_values

def get_quartile_length(mesh_vertices, next_vertex_dict):
    """ A function to find 1/4 of the length of all distances around the boundary of a mesh. 
        Inputs:
        mesh_vertices: array of all vertices of a trimesh.
        next_vertex_dict: a dictionary to find the next vertex on the boundary.  

        Outputs:
        quartile_length: 1/4 of the length of total distance around boundary.
        distances: a dictionary to find the distance of a vertex from its previous vertex.
    """
        
    # get first vertex index on boundary
    current_vertex_idx = list(next_vertex_dict.keys())[0]

    # keep track of distances
    distances = {}
    total_dist = 0.0
    for i in range(len(next_vertex_dict)):
        # get currrent vertex values
        current_vertex = mesh_vertices[current_vertex_idx]

        # get next vertex index
        next_vertex_idx = next_vertex_dict[current_vertex_idx]
        # get next vertex value
        next_vertex = mesh_vertices[next_vertex_dict[current_vertex_idx]]

        dist = np.linalg.norm(next_vertex - current_vertex)
        distances[next_vertex_idx] = dist
        total_dist += dist

        # reset current vertex idk
        current_vertex_idx = next_vertex_idx

    quartile_length = total_dist/4


    return quartile_length, distances

def square_boundary(mesh):
    """ A function to map the boundary of a mesh to a square. 
        Inputs:
        mesh: a trimesh mesh. 
        quartile_dist: float value for the 1/4 of the total distance around boundary
        dist_dict: dictionary that maps vertex to a distance from its previous vertex
        
        Outputs: 
        new_boundary_values: an array of boundary values (2 dimensional) on the circle. 
    """

    mesh_vertices = mesh.vertices
    output = np.zeros((len(mesh_vertices), 2))

    range_x = (np.max(mesh_vertices[:, 0]) - np.min(mesh_vertices[:, 0]))/2
    range_y = (np.max(mesh_vertices[:, 1]) - np.min(mesh_vertices[:, 1]))/2
    range_z = (np.max(mesh_vertices[:, 2]) - np.min(mesh_vertices[:, 2]))/2

    # keep the sqaure roughly the same size as the mesh by choosing half of maximum range as unit
    unit = np.max([range_x, range_y, range_z])

    unit_square = np.array([[0.0,0.0], 
                        [unit,0.0], 
                        [unit,unit],  
                        [0.0,unit]])
    
    boundary_vertices, next_vertex_dict, _ = boundary(mesh)
    quartile_dist, dist_dict = get_quartile_length(mesh_vertices, next_vertex_dict)

    # keep track of total_dist for each edge of square 
    total_dist = 0.0
    # get first vertex index on boundary
    starting_vertex_idx = list(next_vertex_dict.keys())[0]
    current_vertex_idx = starting_vertex_idx
    next_vertex_idx = next_vertex_dict[current_vertex_idx]
    # track vertices to remap (include first two vertices in map)
    vertices_to_map = [current_vertex_idx, next_vertex_idx]
    # include their respective distances to current_vertex
    dist = dist_dict[next_vertex_idx]
    dist_list = [0.0, dist]
    for i in range(4): #loop for each side of square
        # track corners
        A = unit_square[i % len(unit_square)]
        B = unit_square[(i + 1) % len(unit_square)]

        # sum up distances on edge until we reach 1/4 of total distace
        # if final edge of square then sum up all remaining distances (as division will never be perfect)
        while(total_dist + dist < quartile_dist or i == 3):
            next_vertex_idx = next_vertex_dict[current_vertex_idx]

            total_dist += dist
            dist_list.append(total_dist)
            vertices_to_map.append(next_vertex_idx)
            current_vertex_idx = next_vertex_idx
            dist = dist_dict[current_vertex_idx]
            # break once back at starting vertex 
            if(current_vertex_idx == starting_vertex_idx):
                break
        # calculate new positions in two dimensions
        output[vertices_to_map] = [d/total_dist * (B - A) + A for d in dist_list]
        
        # update values for next edge of sqaure
        total_dist = dist
        dist_list = [0.0, dist]
        next_vertex_idx = next_vertex_dict[current_vertex_idx]
        vertices_to_map = [current_vertex_idx, next_vertex_idx]
        current_vertex_idx = next_vertex_idx

    new_boundary_values = output[boundary_vertices]
    return new_boundary_values

In [15]:
# This cell exports the mesh with the boundary vertices transformed to the 2d circle. 
# import matplotlib.pyplot as plt
# %matplotlib inline

# face = trimesh.load("Horse.obj")
# xy = np.array(circle_boundary(face))
# x = xy[:, 0]
# y = xy[:, 1]
# plt.scatter(x, y, marker = 'x')
# b_verts, _ = boundary(face)
# face.vertices[b_verts] = np.hstack((xy, -12*np.ones((len(x), 1))))
# face.export("CircleExperiment.obj")

In [6]:
def angle(v0, v1, v2):

    """ A function to work out the angle between two edges. 
        Inputs:
        v0: vertex coordinates at the centre of the edges
        v1: vertex of edge1
        v2: vertex of edge2
        
        returns: angle between edge from v0 to v1 and edge from v0 to v2
    """

    edge1 = v1 - v0
    edge2 = v2 - v0

    return np.arccos(np.clip(np.dot(edge1, edge2)/(np.linalg.norm(edge1)*np.linalg.norm(edge2)), -1, 1))

In [7]:
def uniformLaplaceBeltrami(mesh):

    """ A function that creates the uniform Laplacian matrix for a given mesh. 
        Inputs:
        mesh: a trimesh mesh
        
        Outputs:
        L: sparse csc matrix containing -1 on the diagonal and 1/k on the off diagonal neighbours.
    """

    n = len(mesh.vertices)
    L = scipy.sparse.lil_matrix(-1*np.eye(n))
    neighbours = mesh.vertex_neighbors

    for i in range(n):
        L[i, neighbours[i]] = 1/len(neighbours[i])

    return L.tocsr()

def cotanLaplaceBeltrami(mesh):

    """ A function to find the cotangent discretization of the Laplacian. 
        Inputs:
        mesh: a trimesh mesh
        Outputs:
        L: the cotangent discretization of the Laplacian. 
    """

    vs = mesh.vertices
    fs = mesh.faces
    vert_ns = mesh.vertex_neighbors
    vert_fs = mesh.vertex_faces

    n = len(vs)
    areas = np.zeros(n)
    C = scipy.sparse.lil_matrix(np.zeros((n, n)))
    
    for vertex in range(n):

        # for every vertex find the neighbours and the faces
        faces = vert_fs[vertex][vert_fs[vertex] != -1]

        for neighbour in vert_ns[vertex]:
            # for every neighbour find the faces that are shared with the vertex, and the other vertices on those faces
            faces = [i for i in vert_fs[vertex] if neighbour in fs[i] and i != -1]
            vertices = [i for i in fs[faces].flatten() if not i in [vertex, neighbour, -1]]

            if len(vertices) > 1:
                
                angle1 = angle(vs[vertices[0]], vs[vertex], vs[neighbour])
                angle2 = angle(vs[vertices[1]], vs[vertex], vs[neighbour])
            
            # This is the case where the mesh is not closed
            else:
                angle1 = angle(vs[vertices[0]], vs[vertex], vs[neighbour])
                angle2 = angle1

            # account for division by zero (and near zero)
            if abs(np.tan(angle1)) < 1e-10:
                cot_alpha = 1e10
            else:
                cot_alpha = 1/np.tan(angle1)

            if abs(np.tan(angle2)) < 1e-10:
                cot_beta = 1e10
            else:
                cot_beta = 1/np.tan(angle2)
            
            # Assign the values 
            C[vertex, neighbour] = cot_alpha + cot_beta
            C[vertex, vertex] -= cot_alpha + cot_beta 

            # voronoi areas 
            areas[vertex] += (C[vertex, neighbour] * np.linalg.norm(vs[vertex] - vs[neighbour])**2) / 8

    M_inv = scipy.sparse.diags(1/(2*areas))
    
    L = (M_inv @ C)
    return L.tocsr()

def mvLaplaceBeltrami(mesh):

    """ A function to find the mean value weighted discretization of the Laplace Beltrami.
        Inputs:
        mesh: a trimesh mesh
        Outputs:
        L: the mean value discretization of the Laplacian.
    """

    vs = mesh.vertices
    fs = mesh.faces
    vert_ns = mesh.vertex_neighbors
    vert_fs = mesh.vertex_faces

    n = len(vs)
    L = scipy.sparse.lil_matrix(np.zeros((n, n)))

    for vertex in range(n):
        # for every vertex find the neighbours and the faces 
        faces = vert_fs[vertex][vert_fs[vertex] != -1]

        for neighbour in vert_ns[vertex]:
            # for every neighbour find the faces that are shared with the vertex, and the other vertices on those faces
            faces = [i for i in vert_fs[vertex] if neighbour in fs[i] and i != -1] 
            vertices = [i for i in fs[faces].flatten() if not i in [vertex, neighbour, -1]]

            if len(vertices) > 1:

                angle1 = angle(vs[vertex], vs[vertices[0]], vs[neighbour])/2
                angle2 = angle(vs[vertex], vs[vertices[1]], vs[neighbour])/2
            
            # This is the case where the mesh is not closed
            else:
                angle1 = angle(vs[vertex], vs[vertices[0]], vs[neighbour])/2
                angle2 = angle1/2
            
            # Assign the values 
            L[vertex, neighbour] = (np.tan(angle1) + np.tan(angle2))/np.linalg.norm(vs[vertex] - vs[neighbour])
            L[vertex, vertex] -= (np.tan(angle1) + np.tan(angle2))/np.linalg.norm(vs[vertex] - vs[neighbour])

    return L.tocsr()    

In [8]:
def tutte_embedding(mesh, boundary_function = circle_boundary, LaplaceBeltrami = cotanLaplaceBeltrami):

    """ A function that uses Tutte's embedding method to flatten a mesh
        with given boundary conditions. 
        Inputs: 
        mesh: a trimesh mesh. 
        boundary_function: a function that returns the 2D coordinates for the boundary vertices.
         
        Outputs: 
        flat_mesh: the flattened mesh. 
    """

    flat_mesh = mesh.copy()

    vertices = mesh.vertices
    b_verts, _, _ = boundary(mesh)
    b_vals = boundary_function(mesh)
    L = LaplaceBeltrami(mesh)
    L = L.tolil()
    
    for vert in b_verts:
        L[vert, :] = np.zeros(L.shape[1])
        L[vert, vert] = 1
    
    L = L.tocsc()
    B = np.zeros((len(vertices), 2))
    B[b_verts] = b_vals

    X = scipy.sparse.linalg.spsolve(L, B)
    flat_mesh.vertices = np.hstack((X, np.zeros((X.shape[0], 1))))
    return flat_mesh

def free_boundary(mesh, LaplaceBeltrami = cotanLaplaceBeltrami):

    """ A function that uses Tutte's embedding method to flatten a mesh
        with free boundaries. 
        Inputs: 
        mesh: a trimesh mesh. 
        LaplaceBeltrami: function to calculate the Laplace Beltrami. 
         
        Outputs: 
        flat_mesh: the flattened mesh. 
    """

    flat_mesh = mesh.copy()
    L = LaplaceBeltrami(mesh)

    _, v = scipy.sparse.linalg.eigs(L, 3, which='SM')

    X = np.real(v[:, 1:]) / np.linalg.norm(v[:, 1:], axis = 0)
    flat_mesh.vertices = np.hstack((X, np.zeros((X.shape[0], 1))))

    return flat_mesh

In [9]:
def virtual_vertices(mesh):

    """ A function to add virtual vertices to non-closed meshes.
        This function only works with one open section in a mesh. 

        This functions works by using the boundary edges, and reflecting the vertex that connects
        to the two vertices in the boundary edge across the edge. This then created a second face for
        the edge, and then I connect all of these reflected vertices to each other, and to a "centroid"
        vertex to create a watertight mesh that can be used. 

        Inputs: 
        mesh: a trimesh mesh
        Outputs: 
        extended_mesh: a watertight version of the original trimesh mesh 
        
    """

    # Create mesh copy and find the boundary vertices and boundary edges
    extended_mesh = mesh.copy()
    boundary_verts, next_vertex, boundary_edges = boundary(mesh)
    vs = mesh.vertices
    neighbs = mesh.vertex_neighbors
    fs = mesh.faces
    normals = mesh.vertex_normals
    n = len(vs)
    virt_faces = [] # this will be a list of virtual faces to add
    virt_verts = [] # this will be a list of virtual vertices to add
    virt_vert_faces = [[i] for i in boundary_verts] # this will be the virtual faces that join only virtual vertices
    
    for edge in boundary_edges:

        index = [i for i in neighbs[edge[0]] if i in neighbs[edge[1]]][0]
        point = vs[index]
        
        # for every boundary face, reflect the inner vertex across the boundary edge
        A_point = point - vs[edge[0]]
        AB = (vs[edge[1]] - vs[edge[0]])/np.linalg.norm((vs[edge[1]] - vs[edge[0]]))
        t = np.dot(A_point, AB)
        projected_point = vs[edge[0]] + t*AB
        projected_point = 2*projected_point - point

        # # create new faces with the new vertices
        indices = [np.where(boundary_verts == edge[0])[0][0], np.where(boundary_verts == edge[1])[0][0]]

        if len(virt_vert_faces[indices[0]]) < 3:
            virt_vert_faces[indices[0]] += [n + len(virt_verts)]
        if len(virt_vert_faces[indices[1]]) < 3:   
            virt_vert_faces[indices[1]] += [n + len(virt_verts)]

        if next_vertex[edge[0]] != edge[1]:
            virt_faces += [[edge[0], edge[1], n + len(virt_verts)]]
        else:
            virt_faces += [[edge[1], edge[0], n + len(virt_verts)]]
            
        virt_verts += [projected_point]

        true_normal = (normals[edge[0]] +  normals[edge[1]] + normals[index])/3
        for i in indices:
            if len(virt_vert_faces[i]) == 3:
                v1, v2, v3 = virt_vert_faces[i]
                normal1 = np.cross((virt_verts[v2 - n] - vs[v1]), (virt_verts[v3 - n] - virt_verts[v2 - n]))
                if np.dot(normal1, true_normal) < 0:
                    virt_vert_faces[i] = [v2, v1, v3]
        
    virt_faces = np.array(virt_faces)
    extended_mesh.vertices = np.vstack((vs, virt_verts))
    extended_mesh.faces = np.vstack((fs, virt_faces, virt_vert_faces))

    return extended_mesh 

In [10]:
def export_textured(mesh_filename, export_name, flat_export_name, bound_func=circle_boundary, texture_file='textures/CheckTexture.png', LB=cotanLaplaceBeltrami):

    """ A function to export a mesh and a flattened mesh with a texture using the 
        flat mesh as the UV parametrization. 
        
        Inputs:
        mesh_filename: mesh to import. 
        export_name: name that the original mesh (with texture) will be saved to. 
        flat_export_name: flattened textured mesh export name. 
        texture_name: optional argument to determine which texture to apply. 
    """

    mesh = trimesh.load(mesh_filename)
    n = len(mesh.vertices)
    flat_mesh = tutte_embedding(mesh, boundary_function=bound_func, LaplaceBeltrami=LB)

    uv_coordinates = flat_mesh.vertices[:, :2]
    texture = Image.open(texture_file)

    mesh.visual = trimesh.visual.TextureVisuals(uv=uv_coordinates)
    flat_mesh.visual = trimesh.visual.TextureVisuals(uv=uv_coordinates)

    mesh.visual.material.image = texture
    flat_mesh.visual.material.image = texture

    mesh.export(export_name, include_texture=True)
    flat_mesh.export(flat_export_name, include_texture=True)

    return 

In [73]:
export_textured("meshes/Face.obj", "c_meshes/uniformRhino.obj", "c_meshes/texFlatRhino1.obj", bound_func=square_boundary, LB = uniformLaplaceBeltrami)
export_textured("meshes/Face.obj", "c_meshes/cotanRhino.obj", "c_meshes/texFlatRhino2.obj", bound_func=square_boundary, LB = cotanLaplaceBeltrami)
export_textured("meshes/Face.obj", "c_meshes/mvRhino.obj", "c_meshes/texFlatRhino3.obj", bound_func=square_boundary, LB = mvLaplaceBeltrami)

In [2]:
import matplotlib as mpl
import matplotlib.cm as cm
def scalarColour(vector):

    """ A function to convert scalar values to RGB float values. 
    """

    norm = mpl.colors.Normalize()
    cmap = cm.viridis
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    return (255*m.to_rgba(vector)).astype('uint8')

camel = trimesh.load("meshes/camel.obj")

# Function to find a "high Gaussian curvature" path between given two points
gc = trimesh.curvature.discrete_gaussian_curvature_measure(camel, camel.vertices, 1)
indices = np.where(gc > np.percentile(gc, 95))[0]
camel.visual.vertex_colors[indices, :] = scalarColour(gc[indices])
camel.export("GC_camel.obj")
# def path(mesh, vertex1, vertex2):


'# https://github.com/mikedh/trimesh\nv -0.00116900 36.48586300 44.32489800 0.40000000 0.40000000 0.40000000\nv 1.33022000 36.76913800 43.79515800 0.40000000 0.40000000 0.40000000\nv 1.48552800 36.13785600 43.29405200 0.40000000 0.40000000 0.40000000\nv -1.48779400 36.13785600 43.29387300 0.40000000 0.40000000 0.40000000\nv -1.33252200 36.76913800 43.79503600 0.40000000 0.40000000 0.40000000\nv 4.26718900 -31.83170700 -29.61946300 0.20784314 0.36470588 0.54901961\nv 4.21880600 -32.38361700 -29.71237000 0.20784314 0.36470588 0.54901961\nv 4.67045400 -31.97112300 -29.78591200 0.27843137 0.17254902 0.48235294\nv 8.28306600 -32.37685000 -21.71886800 0.40000000 0.40000000 0.40000000\nv 8.83510600 -32.50859500 -21.40730300 0.26666667 0.00392157 0.32941176\nv 8.88383700 -32.89292100 -21.42684400 0.14901961 0.49803922 0.55686275\nv 8.66955400 -30.90498000 -24.95654500 0.40000000 0.40000000 0.40000000\nv 8.05289800 -32.12731900 -26.65876000 0.40000000 0.40000000 0.40000000\nv 6.71109800 -32.887

In [109]:
import networkx as nx
import itertools

def curvature_weights(mesh):

    edges = mesh.edges_unique
    length = mesh.edges_unique_length

    gc = trimesh.curvature.discrete_gaussian_curvature_measure(mesh, mesh.vertices, 1)
    ws = np.zeros(len(edges))
    for edge in range(len(edges)):
        ws[edge] = gc[edges[edge][0]] - gc[edges[edge][1]]

    length = ws.max() * (length - length.min())/(length.max() - length.min())
    ws = ws + length
    
    return np.where(ws < 0, 0, ws)

def waypoints(mesh, percent = 2, threshold = 1):

    verts = mesh.vertices
    n = len(verts)
    gc = trimesh.curvature.discrete_gaussian_curvature_measure(mesh, mesh.vertices, 1)
    indices = np.where(gc > np.percentile(gc, 100 - percent))[0].tolist()
    # Only keep points that are not close too together
    indices_copy = indices.copy()
    for i in indices_copy:
        distances = [np.linalg.norm(verts[i] - verts[j]) for j in indices if j != i]
        if np.any(np.array(distances) < threshold):
            indices.remove(i)

    g = nx.Graph()
    for i in indices:
        count = 0
        for j in indices:
            if j == i:
                continue
            if count == 0:
                g.add_edge(i, j, weight=np.linalg.norm(verts[i] - verts[j]))
            else:
                g.add_edge(j, i, weight=np.linalg.norm(verts[i] - verts[j]))
            count += 1

    tree = nx.minimum_spanning_tree(G = g, weight ='weight')
    tree_degrees = {node:val for (node, val) in tree.degree()}

    current_node = max(tree_degrees, key=tree_degrees.get)
    current_cycle = []
    cycles = []
    unvisited = [i for i in tree.edges]
    unvisited = unvisited + [(i[1], i[0]) for i in tree.edges]
    while len(unvisited) > 0:
        
        neighbs = list(tree.neighbors(current_node))
        new_neighbs = []
        for j in range(len(neighbs)):

            if (current_node, neighbs[j]) in unvisited:
                new_neighbs = new_neighbs + [neighbs[j]]
  
        if len(new_neighbs) == 0:
            current_node, next_node = unvisited[0]
            cycles = cycles + [current_cycle]
            current_cycle = []
        
        else:
            next_node = new_neighbs[0]

        current_cycle = current_cycle + [[current_node, next_node]]
        unvisited.remove((current_node, next_node))
        
        current_node = next_node
    
    cycles = cycles + [current_cycle]
    return cycles


def shortest_path(mesh):

    cycles = waypoints(mesh)
    edges = mesh.edges_unique
    weights = curvature_weights(mesh)

    final_path = []

    g = nx.Graph()
    for edge, w in zip(edges, weights):
         g.add_edge(*edge, weight=w)

    i = 0
    print(f"Beginning first of {len(cycles)} cycles.")
    for cycle in cycles:
        cycle_path = []
        for edge in cycle:
            start, stop = edge
            path = nx.dijkstra_path(g, source=start, target=stop, weight = 'weight')
            i += 1
            cycle_path += path
        
        final_path += [cycle_path]
        print("Finished a cycle")

    return final_path

# camel = trimesh.load("meshes/camel.obj")
# shortest_path(camel)


In [110]:
camel = trimesh.load("meshes/camel.obj")
path = shortest_path(camel)
colors = scalarColour(np.linspace(0, len(path), len(path)))
for i in range(len(path)):
    camel.visual.vertex_colors[path[i]] = colors[i, :]

camel.export("paths.obj")
camel.show()

Beginning first of 6 cycles.
Finished a cycle
Finished a cycle
Finished a cycle
Finished a cycle
Finished a cycle
Finished a cycle
